# Main 1 - EMT network analysis

Please refer to the Method section.

In [1]:
import numpy as np
import pandas as pd
import itertools
import networkx as nx
import copy
import os

from pyboolnet.file_exchange import bnet2primes, primes2bnet
from pyboolnet.interaction_graphs import primes2igraph
from pyboolnet.state_transition_graphs import primes2stg
from pyboolnet.attractors import compute_attractors_tarjan

from modules.attractorSim import rand_initial_states, compute_attractor_from_primes, compute_phenotype, Simulation
from modules.stability import compute_networkStability

In [2]:
network_dir = './network/'
model_file = network_dir + 'EMT_Network.bnet'
primes = bnet2primes(model_file)
nodeList = list(primes.keys())
graph = primes2igraph(primes)
update_mode = "synchronous"
   

phenotype = {'E':{'Ecadherin':1, 'ZEB1':0}, 'M':{'Ecadherin':0, 'ZEB1':1}, 'H1':{'Ecadherin':1, 'ZEB1':1}, 'H2':{'Ecadherin':0, 'ZEB1':0}}
phenotypeAnnot = {'E':-1,'M':1, 'H1':0, 'H2':0}
markers = ['Ecadherin','miR200','miR34','Snail','Twist1','ZEB1','EpCAM','THY1','MYC']


if 2**len(nodeList) >= 100000: num_init = 100000
else:  num_init = 2**len(nodeList)
initState = rand_initial_states(num_init, len(nodeList))

### Perturbation analysis

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from collections import defaultdict


def phenotypeAnnot_diff(x):
    if x>=0.5: return (-1)
    elif x<(-0.5): return (1)
    else: return (0)

def perturb_analysis(perturb_p, perturb_s, save_perturbname):
    perturb_p.loc[:,'Desired1'] = [1,1,1,0,0,0,1,0,0] #rEMT
    perturb_p.loc[:,'Desired2'] = [1,1,1,0,0,0,0,0,0] #chemosensitive rEMT
        
    dT1 = perturb_p.loc[:,sorted(set(perturb_p.columns)-set(['Desired2']))]
    didx1 = list(dT1.columns).index('Desired1')
    dc_T1_ = cosine_similarity(dT1.T)
    dT1.loc['cosineSim1',:] = dc_T1_[didx1,:]
    
    dT2 = perturb_p.loc[:,sorted(set(perturb_p.columns)-set(['Desired1']))]
    didx2 = list(dT2.columns).index('Desired2')
    dc_T2_ = cosine_similarity(dT2.T)
    dT2.loc['cosineSim2',:] = dc_T2_[didx2,:]
    
    dT3 = pd.concat([dT1.loc['cosineSim1',:], dT2.loc['cosineSim2',:]], axis=1, sort=True).T
    perturb_p.loc['cosineSim1',:] = dT3.loc['cosineSim1',perturb_p.columns]
    perturb_p.loc['cosineSim2',:] = dT3.loc['cosineSim2',perturb_p.columns]
    perturb_p = perturb_p.fillna(0)
    perturb_p.loc['Diff',:] =  perturb_p.loc['Ecadherin',:] -  perturb_p.loc['ZEB1',:]
    perturb_p.loc['Diff_pheno',:] = [phenotypeAnnot_diff(x) for x in (perturb_p.loc['Ecadherin',:] -  perturb_p.loc['ZEB1',:]).values]    
    
    ## rEMT score
    perturb_p.loc['weight',:] = (perturb_p.loc['Diff',:] + 1)/2
    perturb_p.loc['rEMTscore',:] = perturb_p.loc['cosineSim1',:].values * perturb_p.loc['weight',:].values
    perturb_p.loc['chemo_rEMTscore',:] = perturb_p.loc['cosineSim2',:].values * perturb_p.loc['weight',:].values
    
    # save result
    perturb_result = pd.concat([perturb_p, perturb_s], sort=True).T        
    perturb_result.to_csv(save_perturbname)    
    
    
def perturbAnalysis(allperturbs, fix_dict, save_dir, save_perturbname):
    perturb_p = pd.DataFrame([])
    perturb_s = pd.DataFrame([])
    for perturb in allperturbs:
        fix_dict_tmp = copy.deepcopy(fix_dict)
        fix_dict_tmp.update(perturb)
        print(fix_dict_tmp)
        primes_new, pheno_df, att_ave_pd, attrs_dict = Simulation(fix_dict_tmp, primes, update_mode, initState, phenotype, phenotypeAnnot)
        dT = pd.DataFrame([0 for _ in markers], index = markers, columns = [str(list(perturb.items()))])
        dT.loc[markers,:] = att_ave_pd.loc[markers,:].values
        perturb_p = pd.concat([perturb_p,dT],axis=1, sort=True)     
    
        dS = pd.DataFrame.from_dict(compute_networkStability(attrs_dict, graph, nodeList)[1],orient='index', columns = [str(list(perturb.items()))] )
        perturb_s = pd.concat([perturb_s,dS],axis=1, sort=True)     
    perturb_analysis(perturb_p, perturb_s, save_perturbname)
    

In [7]:
fix_dict = {'TGFb':0,'RAS':1}
save_dir = './result/TGFOFF/' 
if save_dir not in os.listdir(): os.mkdir(save_dir)
save_perturbname = save_dir + 'TGFOFF_single_simul_result.csv'
allsingles = [{n:1} for n in nodeList] + [{n:0} for n in nodeList]

perturbAnalysis(allsingles, fix_dict, save_dir, save_perturbname)

{'TGFb': 0, 'RAS': 1, 'AKT': 1}
WARNING The state transition graph will consist of up to 2**31=2147483648 states, depending on the initial states.
Attractor simulation time : 15.326167106628418
              Ratio
phenotype          
E          0.010996
H1         0.989004
{'TGFb': 0, 'RAS': 1, 'AP1': 1}
WARNING The state transition graph will consist of up to 2**31=2147483648 states, depending on the initial states.
Attractor simulation time : 15.028164148330688
              Ratio
phenotype          
E          0.014136
M          0.985864
{'TGFb': 0, 'RAS': 1, 'ERK': 1}
WARNING The state transition graph will consist of up to 2**31=2147483648 states, depending on the initial states.
Attractor simulation time : 13.609905242919922
             Ratio
phenotype         
H1         0.93911
M          0.06089
{'TGFb': 0, 'RAS': 1, 'Ecadherin': 1}
WARNING The state transition graph will consist of up to 2**31=2147483648 states, depending on the initial states.
Attractor simulation time : 1

Attractor simulation time : 19.47661256790161
              Ratio
phenotype          
E          0.094347
H1         0.905653
{'TGFb': 0, 'RAS': 1, 'AP1': 0}
WARNING The state transition graph will consist of up to 2**31=2147483648 states, depending on the initial states.
Attractor simulation time : 14.79959225654602
              Ratio
phenotype          
E          0.013865
H1         0.986135
{'TGFb': 0, 'RAS': 1, 'ERK': 0}
WARNING The state transition graph will consist of up to 2**31=2147483648 states, depending on the initial states.
Attractor simulation time : 17.05735421180725
              Ratio
phenotype          
E          0.712518
H1         0.287482
{'TGFb': 0, 'RAS': 1, 'Ecadherin': 0}
WARNING The state transition graph will consist of up to 2**31=2147483648 states, depending on the initial states.
Attractor simulation time : 15.148959159851074
             Ratio
phenotype         
H1         0.01372
M          0.98628
{'TGFb': 0, 'RAS': 1, 'EpCAM': 0}
WARNING The state 

### Molecular state ambiguity

In [8]:
from modules.frustration import computeFusingT_updated

fix_dict = {'TGFb':0,'RAS':1}
ctrl = {'TGFb':0,'RAS':1}
save_dir = './result/TGFOFF/' 
computeFusingT_updated(allsingles, fix_dict, ctrl, primes, model_file, save_dir+'TGFOFF_single')